In [1]:
#4/13/20

In [2]:
#my imports, to make sure it works fine on Jupyter Notebook
import findspark
findspark.init()
import pyspark as ps # Call this only after findspark

import sys
import warnings
import re
import json
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.context import SQLContext
from pyspark.sql.functions import udf
from pyspark.sql.functions import lit
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from datetime import datetime
import pyspark.sql.types as T 
from pyspark.sql.functions import split, explode

#To get spark. working without throwing a NameError
import findspark
findspark.init()
import pyspark as ps # Call this only after findspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)


# In[ ]:


#ACTUAL FUNCTION

#for renaming the columns
from functools import reduce

#allow us to use SQL Count function for aggregation in groupBY
from pyspark.sql.functions import count

#allow us to read csv to dataframe
import pandas as pd #need Pandas
from pyspark.sql.types import *

In [3]:
from datetime import datetime, timedelta

t = datetime.strptime("2:00", "%M:%S")
delta = timedelta(minutes=t.minute, seconds = t.second)

inp = spark.createDataFrame(
    [
        (123, "ITEM_PLAY_STARTED", '{"category":"Songs","item_name":"Pop Goes the Weasel2.mp3"}',
        "2020-02-07T18:22:29.877Z"), # create your data here, be consistent in the types.
        (123, "ITEM_PLAY_FINISHED", '{"did_reach_end_of_stream":false,"category":"Songs","item_name":"Pop Goes the Weasel2.mp3"}',
        "2020-02-07T18:22:48.695Z"),
        (123, "ITEM_PLAY_STARTED", '{"category":"Songs","item_name":"Pop Goes the Weasel2.mp3"}',
        "2020-02-07T18:23:29.877Z"), # create your data here, be consistent in the types.
        (123, "ITEM_PLAY_FINISHED", '{"did_reach_end_of_stream":false,"category":"Songs","item_name":"Pop Goes the Weasel2.mp3"}',
        "2020-02-07T18:23:48.695Z")
    ],
    ['device_id', 'event', "payload", "time"] # add your columns label here
)


In [4]:
inp.show(truncate = False)

+---------+------------------+-------------------------------------------------------------------------------------------+------------------------+
|device_id|event             |payload                                                                                    |time                    |
+---------+------------------+-------------------------------------------------------------------------------------------+------------------------+
|123      |ITEM_PLAY_STARTED |{"category":"Songs","item_name":"Pop Goes the Weasel2.mp3"}                                |2020-02-07T18:22:29.877Z|
|123      |ITEM_PLAY_FINISHED|{"did_reach_end_of_stream":false,"category":"Songs","item_name":"Pop Goes the Weasel2.mp3"}|2020-02-07T18:22:48.695Z|
|123      |ITEM_PLAY_STARTED |{"category":"Songs","item_name":"Pop Goes the Weasel2.mp3"}                                |2020-02-07T18:23:29.877Z|
|123      |ITEM_PLAY_FINISHED|{"did_reach_end_of_stream":false,"category":"Songs","item_name":"Pop Goes the Weas

In [9]:
def main(inputFile, outputFile, configFile):
    from pyspark.sql.functions import col
    from pyspark.sql.functions import split
    from pyspark.sql.functions import substring
    from pyspark.sql.types import StringType
    from pyspark.sql.functions import udf
    from pyspark.sql.functions import trim
    import pyspark.sql.functions as F

#     df = spark.read.parquet(inputFile+'/*').dropDuplicates().na.drop()
#     contentMapping = spark.read.parquet(contentMapping+'/*') #right formatting?
    df = inputFile
    
    #DF for Item Play Started
    start = df.filter(col('event').isin(['ITEM_PLAY_STARTED']))
    #split json string and get just the name
    start = start.withColumn('Content Name', split(df['payload'], ':')[2])
    name_UDF = udf(lambda x:x[1:-2],StringType()) 
    start = start.withColumn('Content Name', name_UDF('Content Name'))
    #turn Time Start String to easy to use Time Stamp Object
    start = start.withColumn('Time Start', df['time'].cast("timestamp"))
    #Select just the columns we need
    cols = ["device_id", "Content Name", "Time Start"]
    start = start.select(*cols)
    
    
    #DF for Item Play Finished
    finished = df.filter(col('event').isin(['ITEM_PLAY_FINISHED']))
    #split json string and get just the name
    finished = finished.withColumn('Content Name', split(df['payload'], ':')[3])
    finished = finished.withColumn('Content Name', name_UDF('Content Name'))
    
    #split json string and get whether content was played in entirety
    finished = finished.withColumn('reach_end_of_stream', split(df['payload'], ':')[1])
    stream_end_UDF = udf(lambda x:x[0:5],StringType()) 
    finished = finished.withColumn('reach_end_of_stream', stream_end_UDF('reach_end_of_stream'))
    
    #Get rid of white space
    finished = finished.withColumn("reach_end_of_stream", trim(col("reach_end_of_stream")))
    
    #Convert True/False strings to actual boolean values
    finished = finished.withColumn(
    'reach_end_of_stream',
    F.when(F.col("reach_end_of_stream") == "true", True)\
    .otherwise(False)
    )
    #turn Time End String to easy to use Time Stamp Object
    finished = finished.withColumn('Time End', df['time'].cast("timestamp"))
    #Select just the columns we need
    cols = ["device_id", "Content Name", "Time End", "reach_end_of_stream"]
    finished = finished.select(*cols)
    
    #combine two dataframes for our transformed Schema
    transformed = start.join(finished, on=["device_id", "Content Name"], how='left_outer')
    
    #Make sure Time Start before time end
    transformed = transformed.where(col("Time Start") <= col("Time End"))
    
    #Convert time stamps to unix
    #transformed = transformed.withColumn('Time Start', F.unix_timestamp('Time Start'))
    #transformed = transformed.withColumn('Time End', F.unix_timestamp('Time End'))


    
#Find min within Time End list
    temp = transformed.groupby("Time Start").agg(F.collect_list("Time End").alias("Time End List"))#.min("Time End List")
    min_time_UDF = udf(lambda x: __builtins__.min(x), TimestampType()) 
    #***************************************************************************************
    #***This is where it gets incredibly laggy. For some reason min. UDF doesn't work well. 
    #***************************************************************************************
    temp = temp.withColumn('Time End', min_time_UDF('Time End List'))
#     cols = ["Time Start", "Time End"]
#     temp = temp.select(*cols)

    
    #Combine temp and transformed 
    #transformed = transformed.join(temp, on=["Time Start", "Time End"], how='right')
    #transformed = transformed.select("device_id","Content Name","Time Start","city")

    
#     event_loggerino = pd.merge(play_started, play_finished, on = ["device_id", "Content Name"]) #merge two df's together on id/content name
#     event_loggerino = event_loggerino[ ["device_id", "Content Name", "Time Start", "Time End", "Content Consumed Entirely?"] ] #columns wanted in the order I want them
#     event_loggerino["Content Consumed Entirely?"] = event_loggerino["Content Consumed Entirely?"].astype(int) #Convert True/False values to 1/0

#     df.write.parquet(outputFile) # Write onto output Parquet
    return temp.show(truncate = False) #SEE NOTE WRITTEN IN ALL CAPS

In [ ]:
main(inp, "dd", "dd")